In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate as sci
import time
from getdist import plots, MCSamples
import time

import tensorflow as tf
ds = tf.contrib.distributions

%matplotlib inline
dpi = 200

/home/mauricio/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# saved things directory
direc = '/home/mauricio/Documents/Uni/Intro_2/' + 'gal.txt'

# Carga de datos
redshift = np.genfromtxt('gal.txt', usecols=(1))
mu_obs = np.genfromtxt('gal.txt', usecols=(2)) # m - M
cov = np.genfromtxt('gal.txt', usecols=(3))

p = np.argsort(redshift)
redshift = redshift[p]
mu_obs = mu_obs[p]
cov = cov[p]
cov = np.diag(cov)

In [3]:
tf.reset_default_graph()

def f1(theta, z, omk):
    zc = np.copy(z)
    zc = np.insert(zc, 0, 0)
    dz = zc[1:] - zc[:-1]
    E = EHubble(theta, z)[0]
    I = tf.cumsum(dz/(E + 1e-300))
    o_k_s = tf.sqrt(abs(omk))
    return (1 + z)*tf.sinh(o_k_s*I)/(o_k_s + 1e-300)


def f2(theta, z, omk):
    zc = np.copy(z)
    zc = np.insert(zc, 0, 0)
    dz = zc[1:] - zc[:-1]
    E = EHubble(theta, z)[0]
    I = tf.cumsum(dz/(E + 1e-300))
    o_k_s = tf.sqrt(abs(omk))
    return (1 + z)*tf.sin(o_k_s*I)/(o_k_s + 1e-300)


def f3(theta, z, omk):
    zc = np.copy(z)
    zc = np.insert(zc, 0, 0)
    dz = zc[1:] - zc[:-1]
    E = EHubble(theta, z)[0]
    I = tf.cumsum(dz/(E + 1e-300))
    return (1 + z)*I


def EHubble(theta, z): # parametro de hubble
    om0 = theta[0]
    ol = theta[1]
    w = theta[2]
    arg = om0*(1 + z)**3 + (1 - om0 - ol)*(1 + z)**2 + ol*(1 + z)**(3*(1 + w))
    EE = tf.sqrt(arg)
    return EE, arg


def modelo(theta, z):
    om0 = theta[0]
    ol = theta[1]
    w = theta[2]    
    omega_k = 1 - om0 - ol
    # integral    
    dl = tf.case({tf.greater(omega_k, 0.): lambda: f1(theta, z, omega_k), 
                 tf.less(omega_k, 0.): lambda: f2(theta, z, omega_k)}, 
                default=lambda: f3(theta, z, omega_k), exclusive=True)
    dist = 5*tf.log(dl + 1e-300)/np.log(10)
    return dist


class potential:
    def __init__(self, dat, sigma, z):
        self.data = dat
        self.cov = sigma
        self.z = z
    
    def value(self, theta):
        self.mod = modelo(theta, self.z)
        self.u = - likelihood(self.mod, self.data, self.cov) #- prior(theta, ndim) 
        return self.u
    
    def grad(self, theta):
        self.gradient = tf.gradients(self.u, theta)
        return self.gradient


def likelihood(mod, dat, sigma): # retorna escalar, log(L)
    """Log likelihood
    mod: tf.tensor with model results
    dat: numpy array of data
    sigma: numpy array of covariance"""
    sig = tf.cast(tf.diag_part(sigma), tf.float32)
    L = -0.5*chi2(mod, dat, sigma)[0]  + tf.reduce_sum(-0.5*tf.log(2*np.pi*sig**2))
    return L


def chi2(mod, dat, sigma):
    sig = tf.cast(tf.diag_part(sigma), tf.float32)
    AA = tf.reduce_sum(tf.square((dat - mod)/sig))
    BB = tf.reduce_sum((dat - mod)/tf.square(sig))
    CC = tf.reduce_sum(1/tf.square(sig))
    chi = AA - (BB**2)/CC
    return chi, BB/CC

In [4]:
class prior:
    def __init__(self, name, low=None, high=None, mean=None, cov=None):
        if name=='uniform':
            self.u = tf.distributions.Uniform(low=low, high=high)
        elif name=='normal':
            self.u = tf.contrib.distributions.MultivariateNormalFullCovariance(loc=mean, covariance_matrix=cov)
    
    def get_samples(self, n):
        return self.u.sample(sample_shape=(n))
    
    def get_pdf(self, value):
        return self.u.prob(value)
    
    def get_log_pdf(self, value):
        return self.u.log_prob(value)

In [28]:
def time_encodig(m, m_act):
    arg = 2*np.pi*m_act/m
    val = np.array([np.cos(arg), np.sin(arg)])
    return tf.assign(t1, val)


class Leapfrog:
    def __init__(self, U, m, ndim, nnx, nnv, e, b):
        """
        U: potential energy function
        m: # of leapfrog steps
        ndim: # of dimensions
        nnx: neural network of x's
        nnv: neural network of v's
        e: leapfrog step parameter
        b: batch size
        """
        self.U = U
        self.m = m
        self.ndim = ndim
        self.nnx = nnx
        self.nnv = nnv
        self.e = e
        self.b = b
        
    def direction(self):
        self.d = np.random.choice([-1, 1])
        
    def for_dyn(self, x0, v0):
        """One step of forward dynamics d=1.
        x0: actual position in parameter space.
        v0: actual velocity.
        """
        # remember S, Q, T update in each sub iteration
        self.v = v0*tf.exp(0.5*self.nnv.S*self.e) - 0.5*self.e*(self.U.grad(x0)*tf.exp(self.nnv.Q*self.e) 
                                                                + self.nnv.T)
        self.x = x0*tf.exp(self.e*self.nnx.S) + self.e*(self.v*tf.exp(self.e*self.nnx.Q) + self.nnx.T)
        self.v = self.v*tf.exp(0.5*self.nnv.S*self.e) - 0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) 
                                                                    + self.nnv.T)
    
    def back_dyn(self, x0, v0):
        """One step of backward dynamics d=-1
        x0: actual position in parameter space.
        v0: actual velocity.
        """
        self.v = v0*tf.exp(- 0.5*self.nnv.S*self.e) + 0.5*self.e*(self.U.grad(x0)*tf.exp(self.nnv.Q*self.e) 
                                                                + self.nnv.T)
        self.x = x0*tf.exp(- self.e*self.nnx.S) - self.e*(self.v*tf.exp(self.e*self.nnx.Q) + self.nnx.T)
        self.v = self.v*tf.exp(- 0.5*self.nnv.S*self.e) + 0.5*self.e*(self.U.grad(self.x)*tf.exp(self.nnv.Q*self.e) 
                                                                    + self.nnv.T)
        
    def dyn(self, x0, v0):
        """Dynamics.
        x0: inital position in parameter space
        v0: inital velocity.
        """
        self.direction()
        for i in range(self.m):
            #self.mask = np.random.choice([1, 0], size=(self.b, self.ndim), p=[0.5, 0.5])
            #self.nmask = np.ones((self.b, self.ndim)) - self.mask
            if self.d==1:
                self.for_dyn(x0, v0)
            elif self.d==-1:
                self.back_dyn(x0, v0)
            x0 = self.x
            v0 = self.v
        return self.x, self.v
        
    def resample(self):
        self.v = tf.random_normal(shape=(self.b, self.ndim), mean=0.0, stddev=1.0, dtype=tf.float32)
        self.direction()
    
    def flip(self):
        self.d *= -1
    
        
class MLP:
    def __init__(self, ndim, n1, n2, ls, lq):
        """
        ndim: # of dimensions
        n1: # of neurons of layer 1
        ls: output parameter
        lq: output parameter
        """
        self.W1 = tf.Variable(tf.random_normal([ndim, n1]))
        self.W2 = tf.Variable(tf.random_normal([ndim, n1]))
        self.W3 = tf.Variable(tf.random_normal([2, n1])) # time encoding
        self.W4 = tf.Variable(tf.random_normal([n1, n2]))
        self.Ws = tf.Variable(tf.random_normal([n2, 1]))
        self.Wq = tf.Variable(tf.random_normal([n2, 1]))
        self.Wt = tf.Variable(tf.random_normal([n1, 1]))
        self.b1 = tf.Variable(tf.random_normal([n1]))
        self.b2 = tf.Variable(tf.random_normal([n2]))
        self.bs = tf.Variable(tf.random_normal([ndim]))
        self.bq = tf.Variable(tf.random_normal([ndim]))
        self.bt = tf.Variable(tf.random_normal([ndim]))
        self.ls = ls
        self.lq = lq
        
    
    def model(self, x, v, t):
        h1 = tf.matmul(x, self.W1) + tf.matmul(v, self.W2) + tf.matmul(t, self.W3) + self.b1
        h1 = tf.nn.relu(h1)
        h2 = tf.matmul(h1, self.W4) + self.b2
        h2 = tf.nn.relu(h2)
        self.S = tf.tanh(tf.matmul(h2, self.Ws) + self.bs)
        self.Q = tf.tanh(tf.matmul(h2, self.Wq) + self.bq)
        self.T = tf.matmul(h2, self.Wt) + self.bt
    
    
    #loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    #train_op = optimizer.minimize(loss_op)

class L2HMC:
    def __init__(self, U, prior, n, b, m, lr, sc, reg):
        """
        U: energy function
        prior: prior distribution
        n: # iterations
        b: batch size
        m: leapfrog steps
        lr: learning rate
        sc: scale parameter
        reg: regularization parameter
        """
        self.U = U
        self.prior = prior
        self.n = n
        self.b = b
        self.m = m
        self.lr = lr
        self.sc = sc
        self.reg = reg
        self.init_samples = self.prior.get_samples(b)
        
    def run(self):
        for i in range(self.n):
            #print(i)
            self.mini_batch = self.prior.get_samples(self.b)
            for j in range(self.b):
                pass
                #print(j)
                


In [25]:
low = [0., 0., -6.]
high = [1., 1., 1/3]
batch_size = 5

In [26]:
pot = potential(mu_obs, cov, redshift)
pr = prior(name='uniform', low=low, high=high)
l2 = L2HMC(pot, pr, 10, batch_size, 2, 1e-3, 1e-1, 1e-1)
mlpx = MLP(ndim=3, n1=10, n2=10, ls=1, lq=1)
mlpv = MLP(ndim=3, n1=10, n2=10, ls=1, lq=1)


x1 = np.array([[0.5, 0.4, -0.5], [0.5, 0.4, -0.5]])
x1 = tf.convert_to_tensor(x1, dtype=tf.float32)

v1 = np.array([[0.3, 0.4, -1.5], [0.3, 0.4, -1.5]])
v1 = tf.convert_to_tensor(v1, dtype=tf.float32)

t1 = np.zeros((2, 2))
t1 = tf.convert_to_tensor(t1, dtype=tf.float32)

a1 = np.array([1, 2, 3])
a1 = tf.convert_to_tensor(a1, dtype=tf.float32)

mlpx.model(x1, v1, t1)

t2 = x1*a1

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(x1))
    print(sess.run(a1))
    print(sess.run(t2))
l2.run()

[[ 0.5  0.4 -0.5]
 [ 0.5  0.4 -0.5]]
[1. 2. 3.]
[[ 0.5  0.8 -1.5]
 [ 0.5  0.8 -1.5]]


In [41]:
lp = Leapfrog(U=pot, m=5, ndim=3, nnx=mlpx, nnv=mlpv, e=1e-3, b=batch_size)
lp.resample()
print(lp.d)
print(lp.v)

-1
Tensor("random_normal_169:0", shape=(5, 3), dtype=float32)
